In [66]:
import openai

def load_api_key(path: str = '../.env') -> None:
    with open(path,'r') as keyfile:
        openai.api_key = keyfile.read().strip('\n').strip().split('=')[1]
load_api_key()

In [383]:
from typing import Protocol, Required, Any
from functools import reduce
import json 

from pydantic.dataclasses import dataclass, Field
from pydantic.main import ModelMetaclass
from pydantic import create_model

from enum import StrEnum, auto
from typing import TypedDict
import itertools


class ChatRoles(StrEnum):
    """Chat response roles in chat gpt api"""
    system = auto()
    user = auto()
    assistant = auto()
    
class FormattedChatMessage(TypedDict):
    role: ChatRoles.user
    content: str
    
    
@dataclass
class ChatMessage:
    """Chat message content in chat gpt api"""
    role: ChatRoles = ChatRoles.user
    content: str = ""
    
    def __post_init__(self):
        if self.content == "":
            raise ValueError("Content should not be empty, else we waste tokens!")
            
    def _minify_prompt(self) -> str:
        return (self.content.replace('  ','').strip()   # remove visual indent
            .replace("\n", " ").strip()   # remove newlines
            .replace("<br>","\n"))  # add newlines for paragraph breaks only.
    
    def formatted(self) -> FormattedChatMessage:
        return {"role": self.role.value, "content": self._minify_prompt()}
    
USER_DEFAULT_PROMPT = """
    You are going to give detailed helpful suggestions to improve the cohesion, readability,
    and ease of use of a proposed SQL database table schema. I will provide a JSON formatted prompt
    starting with a table name, a one-sentence description of the contents of the table
    and what the expected usecase for the data that will be uploaded to this table.
    The usecases are usually analytics or diagnostics for data uploaded by a set of devices sold by a business.
    The prompt will continue with a list of proposed column names, data type of the column,
    and description of the content. <br>
    
    Using your expertise in database management, data analysis,
    and speaking as someone who must ensure that the names of data fields are consistent,
    descriptive and easy to understand, you will propose helpful changes to the JSON formatted
    table schema, while concisely explaining your reasoning behind your proposed changes. You
    must answer in a concise and space efficient way, providing deeper reasoning if later asked. <br>
    
    When proposing changes, you will ensure that the column names use a uniform pythonic
    format with underscores "_" to connect words and mention the units of the measurement
    if relevant. For example, columns counting something should end with "_counts", a time measurement
    should include "_seconds" or "_ms" depending on the data measurement unit. <br>
    
    You will spend the most effort making absolutely sure that the column names are descriptive
    and specific, the column names are easy to read and understand by users who may
    not be familiar with the underlying device or data, in general are easy to read and understand at a glance,
    and that they are uniform in style, formatting, structure, and follow a uniform naming convention.
    This means that the column names and descriptions are not overly generic, unclear, or unspecific.
    They do not use different words to refer to the same things.
    You will also check the spelling and grammar of the descriptions of the columns. <br>
    
    If you find that you need to propose a large amount of changes, you will limit yourself to only
    proposing what to change, without explaining the reasoning behind it. If you find that the prompt with
    the table columns and descriptions is already very good with a uniform style and clear naming conventions,
    you can explain more of your reasoning since you will only propose minimal changes.
"""

ASSISTANT_DEFAULT_RESPONSE = """Sure, I can help with improving the SQL schema.
    Please provide me with the table name, description, and column names with data type and description,
    in JSON format, so I can propose useful suggestions for improving the SQL schema."""

def compose_prompt(*messages: list[ChatMessage]) -> list[FormattedChatMessage]:
    _messages = itertools.chain.from_iterable(messages)
    return [message.formatted() for message in _messages]

    
def get_init_prompt() -> list[ChatMessage]:
    system = ChatMessage(
        role=ChatRoles.system, 
        content="You are a helpful assistant. Answer as concisely as possible.")
    
    user = ChatMessage(
        role=ChatRoles.user,
        content=USER_DEFAULT_PROMPT)
    
    assistant = ChatMessage(
        role=ChatRoles.assistant,
        content=ASSISTANT_DEFAULT_RESPONSE)
    
    return [system,user,assistant]


class ColumnSpecification(TypedDict):
    type: str
    description: str
    
    
@dataclass
class TemplateTable:
    col1_name: str = Field(description='col1 description text.', default='')
    col2_name: str = Field(description='col2 description.', default='')
    
    # Raise error if someone tries to instantiate the template..
    def __post_init__(self):
        raise NotImplementedError("This is just a template for reference!")
    

class Contract(TypedDict):
    table: str
    table_description: str
    columns: dict[str|ColumnSpecification]
    
    
def to_snake_case(str) -> str:
    """Convert CamelCase to snake_case (well actually `camel_case`)."""
    return reduce(lambda x, y: x + ('_' if y.isupper() else '') + y, str).lower()
    
class PydanticSchema(Protocol):
    __pydantic_model__: ModelMetaclass
    
class HasPydanticSchema(Protocol):
    """Interface for pydantic schema"""
    @staticmethod
    def schema(by_alias: bool, ref_template: 'unicode') -> dict[str | Any]:
        ...



KNOWN_TYPES = {
    "int": int,
    "integer": int,
    "float": float,
    "number": float,
    "str": str,
    "string": str,
}

PYDANTIC_TYPES = {
    int: "integer",
    float: "float",
    str: "string",
}

def type_convert(type_str: str) -> str:
    if type_str in KNOWN_TYPES:
        return PYDANTIC_TYPES[KNOWN_TYPES[type_str]]
    return type_str
        


def dynamically_create_pydantic_model(table_name: str, fields=list[tuple[str, type | str, str]]) -> HasPydanticSchema:
    """Create a pydantic model table from dynamically input fields."""
    return create_model(
        table_name,
        **{
            name:(KNOWN_TYPES[typ], Field(description=desc))
            for (name, typ, desc) in fields
        }
    )


def make_contract(table: HasPydanticSchema, description: str = "This is a table with data.") -> Contract:
    schema = table.schema()
    table_name = {'table': to_snake_case(schema['title']),
                 'table_description': description}
    columns = {field_name: {"type": type_convert(typ_format), "description": properties["description"]}
               for field_name, properties in schema["properties"].items()
               if (typ_format := properties.get('format', properties['type'])) is not None }
    
    return table_name | {"columns": columns}

class ChatModels(StrEnum):
    gpt_turbo = 'gpt-3.5-turbo'
    davinci_02 = 'text-davinci-002'
    code_davinci = 'code-davinci-002'  # 8000 tokens!
    
def make_final_prompt(contract: Contract, *additional_prompts: ChatMessage) -> list[FormattedChatMessage]:
    table_schema_prompt = ChatMessage(content=json.dumps(contract))
    final_prompt = compose_prompt(get_init_prompt(), [table_schema_prompt], additional_prompts)
    return final_prompt

def ask_chatgpt(prompt: list[FormattedChatMessage], model: ChatModels = ChatModels.gpt_turbo) -> openai.openai_object.OpenAIObject:
    completion = openai.ChatCompletion.create(
        model=model,
        messages=prompt
    )
    return completion

def get_feedback(contract: Contract, model: ChatModels = ChatModels.gpt_turbo) -> openai.openai_object.OpenAIObject:
    final_prompt = make_final_prompt(contract)
    return ask_chatgpt(final_prompt)

def pretty_print_response(feedback: openai.openai_object.OpenAIObject):
    print(feedback.choices[0].message.content)
    return feedback.choices[0]

def pydantic_model_from_contract(contract: Contract) -> HasPydanticSchema:
    fields = [(colname, properties['type'], properties['description']) for colname, properties in contract['columns'].items()]
    return dynamically_create_pydantic_model(contract['table'], fields)

In [327]:
@dataclass
class AddStats:
    add_detections: int = Field(description='total number of audio distortions detected.')
    add_detection_class: str = Field(description='Audo distortion detected class.')
    add_classes: str = Field(description="comma separated list of distortion classes. For example: 'Transients, Spikes'.") 
    add_transients_count: int = Field(description="Total number of Transients detected, in counts.") 
    add_spikes_count: float = Field(description="Total number of spikes detected, in counts.")
    
contract = make_contract(AddStats.__pydantic_model__, description="Audio distortion detection stats from the microphones of our cameras.")
#new_contract_feedback = get_feedback(contract)

In [308]:
output = pretty_print_response(new_contract_feedback)

Thank you for the prompt. Here are my proposed changes to improve the schema:

- Rename the table to "audio_distortion_detection_stats" to make it more descriptive and easier to understand at a glance.
- Rename the column "add_detections" to "total_distortion_detections" to be more specific and descriptive.
- Rename the column "add_detection_class" to "distortion_detection_class" to follow a uniform naming convention and be more descriptive.
- Rename the column "add_classes" to "distortion_classes" to follow a uniform naming convention and be clearer in meaning.
- Add a new column called "camera_id" with data type "integer" and description "ID of the camera that generated the audio data" to make it easier to associate data with specific cameras in the future.
- Rename the column "add_transients_count" to "total_transient_counts" to follow a uniform naming convention and be more specific.
- Change the data type of the "add_spikes_count" column from "float" to "integer" since it's unlike

In [187]:
print(contract_feedback.choices[0].message.content)

Thanks for providing the JSON. Here are some suggestions for improving the SQL schema. 

1. Rename the table name to "audio_distortion_detection" to be more descriptive and consistent with the content of the table. 

2. Rename "add_detections" column to "total_detections" to be more specific and easy to understand. 

3. Rename "add_detection_class" to "detection_class" to be less generic and more consistent with column naming.

4. Rename "add_classes" to "distortion_classes" for a more descriptive column name. 

5. Modify the "add_transients_counts" column to "transients_detected" for consistent column naming convention.

6. Change the data type of "add_spikes_count" from number to integer for uniformity and consistency.

7. Add a new column "timestamp" with the data type of "timestamp" to capture the date and time of the audio distortion detection. This will help in data analysis and provide more context for the data.

Overall, these changes aim to make the schema more descriptive, sp

# After fixing!

In [ ]:
import datetime
@dataclass
class AudioDistortionDetectionStats:
    total_distortion_detections: int = Field(description='total number of audio distortions detected.')
    distortion_detection_class: str = Field(description='Audo distortion detected class.')
    distortion_classes: str = Field(description="Comma separated list of distortion classes. For example: 'Transients, Spikes'.") 
    total_transients_count: int = Field(description="Total number of Transients detected, in counts.") 
    total_spikes_count: int = Field(description="Total number of spikes detected, in counts.")
    timestamp: str = Field(description="String formatteed timestamp at date and time of the distortion detection.")
    camera_id: int = Field(description="ID of the camera that generated the audio data.")
    
fixed_contract = make_contract(AudioDistortionDetectionStats.__pydantic_model__, 
                               description="Audio distortion detection stats from the microphones of our cameras.")

In [356]:
fixed_contract_feedback = get_feedback(fixed_contract)
_ = pretty_print_response(fixed_contract_feedback)

Here are my suggestions for improving the SQL schema for the "audio_distortion_detection_stats" table:

1. Change the column name "total_distortion_detections" to "total_distortion_detections_counts" to make it more specific and in line with the naming convention suggested in the prompt.
2. Change the column name "distortion_detection_class" to "detected_distortion_class" for consistency in column naming.
3. Change the column name "distortion_classes" to "detected_distortion_classes" for consistency and clarity in column naming.
4. Change the column name "total_transients_count" to "transients_counts" for consistency in column naming.
5. Change the column name "total_spikes_count" to "spikes_counts" for consistency in column naming.
6. Change the data type of the "timestamp" column from "date-time" to "timestamp" for consistency and clarity in data types.
7. Keep the "camera_id" column name and data type as is since they are already clear and specific.

Overall, these changes will help

In [380]:
import datetime
@dataclass
class AudioDistortionDetectionStats:
    total_distortion_detections: int = Field(description='total number of audio distortions detected.')
    detected_distortion_class: str = Field(description='Audo distortion detected class.')
    detected_distortion_classes: str = Field(description="Comma separated list of distortion classes. For example: 'Transients, Spikes'.") 
    transients_count: int = Field(description="Total number of Transients detected, in counts.") 
    spikes_count: int = Field(description="Total number of spikes detected, in counts.")
    timestamp: datetime.datetime = Field(description="Timestamp at date and time of the distortion detection.")
    camera_id: int = Field(description="ID of the camera that generated the audio data.")
    
fixed_contract_2 = make_contract(AudioDistortionDetectionStats.__pydantic_model__, 
                               description="Audio distortion detection stats from the microphones of our cameras.")


additional_messages = [ChatMessage(role=ChatRoles.assistant, content=fixed_contract_feedback.choices[0].message.content.replace("\n","<br>")),
 ChatMessage(content=f"I fixed it according to your suggestions. Do you see any other mistakes? Please also check spelling and grammar. {json.dumps(fixed_contract_2)}")]

In [381]:
additional_messages[0].formatted()

{'role': 'assistant',
 'content': 'Here are my suggestions for improving the SQL schema for the "audio_distortion_detection_stats" table:\n\n1. Change the column name "total_distortion_detections" to "total_distortion_detections_counts" to make it more specific and in line with the naming convention suggested in the prompt.\n2. Change the column name "distortion_detection_class" to "detected_distortion_class" for consistency in column naming.\n3. Change the column name "distortion_classes" to "detected_distortion_classes" for consistency and clarity in column naming.\n4. Change the column name "total_transients_count" to "transients_counts" for consistency in column naming.\n5. Change the column name "total_spikes_count" to "spikes_counts" for consistency in column naming.\n6. Change the data type of the "timestamp" column from "date-time" to "timestamp" for consistency and clarity in data types.\n7. Keep the "camera_id" column name and data type as is since they are already clear and 

In [382]:
final_prompt = make_final_prompt(fixed_contract, *additional_messages)
fixed_contract_2_feedback = ask_chatgpt(final_prompt)
#fixed_contract_2_feedback = get_feedback(fixed_contract, *additional_messages)
response_2 = pretty_print_response(fixed_contract_2_feedback)

The revised JSON is looking good, and I don't see any further mistakes. The spelling and grammar in the descriptions of the columns are clear and correct, making the schema easier to follow for future users who may not be familiar with the underlying device or data. The changes we've made will help ensure that the column names and descriptions are universally understandable, easy to read and not confusing, and follow a uniform naming convention.


# WOW!

In [392]:
#	$0.002 / 1K tokens
def get_cost(tokens: int, cost: float = 0.002/1000) -> float:
    return tokens * 0.002/1000

print(fixed_contract_2_feedback.usage)
cost = get_cost(sum(fixed_contract_2_feedback.usage.values()))
print(f"Cost: ${cost} or SEK {cost*10} kr.")

{
  "completion_tokens": 86,
  "prompt_tokens": 1242,
  "total_tokens": 1328
}
Cost: $0.005312 or SEK 0.05312 kr.


### **Do you store the data that is passed into the API?**


As of March 1st, 2023, we retain your API data for 30 days but
no longer use your data sent via the API to improve our models. 

Learn more in our [data usage policy](https://openai.com/policies/usage-policies).

### Content co-authored with the OpenAI API



Creators who wish to publish their first-party written content (e.g., a book, compendium of short stories) created in part with the OpenAI API are permitted to do so under the following conditions:

    The published content is attributed to your name or company.
    The role of AI in formulating the content is clearly disclosed in a way that no reader could possibly miss, and that a typical reader would find sufficiently easy to understand.
    Topics of the content do not violate OpenAI’s Content Policy or Terms of Use, e.g., are not related to political campaigns, adult content, spam, hateful content, content that incites violence, or other uses that may cause social harm.
    We kindly ask that you refrain from sharing outputs that may offend others.

For instance, one must detail in a Foreword or Introduction (or some place similar) the relative roles of drafting, editing, etc. People should not represent API-generated content as being wholly generated by a human or wholly generated by an AI, and it is a human who must take ultimate responsibility for the content being published.

Here is some stock language you may use to describe your creative process, provided it is accurate:

    The author generated this text in part with GPT-3, OpenAI’s large-scale language-generation model. Upon generating draft language, the author reviewed, edited, and revised the language to their own liking and takes ultimate responsibility for the content of this publication.